# Download Finance Data and Make Simple Calculation

In [ ]:
# Install Dependencies
%pip install -r requirements.txt

In [1]:
# Imports
import numpy as np
import pandas as pd

#Fin Data Sources
import pandas_datareader as pdr

# Yahoo finance
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date


### Question 1: [Macro] Average growth of GDP in 2023

What is the average growth (in %) of GDP in 2023?

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer 5.7

In [2]:
# Define FRED code for GDPC1
fred_code = "GDPC1"

# Download GDPC1 data start at 2010
# For calculating YoY growth, you need at least 4 data points (current and 4 quarters back)
gdp_data = pdr.DataReader(fred_code, "fred", start="2010-01-01", end="2024-04-18")

# Convert index to datetime (optional, pandas_datareader usually handles this)
# gdp_data.index = pd.to_datetime(gdp_data.index)

# Calculate YoY growth rate between the current GDP value and the GDP value from 4 quarters ago (YoY).
yoy_growth = (gdp_data / gdp_data.shift(4)) * 100

# Filter data for 2023 and calculate average YoY growth
yoy_growth_2023 = yoy_growth[yoy_growth.index.year == 2023]
average_growth_2023 = round(yoy_growth_2023.mean(), 1)

# Print the average YoY growth in 2023
print(f"Average YoY Growth Rate (2023): {average_growth_2023}%")


Average YoY Growth Rate (2023): GDPC1    102.5
dtype: float64%


### Question 2. [Macro] Inverse "Treasury Yield"
Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.

Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2, https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

In [3]:
# Define FRED codes for DGS2 and DGS10
dgs2_code = "DGS2"
dgs10_code = "DGS10"

# Download data from FRED 
start_date = "2000-01-01"
end_date = "2024-04-18"  # Adjust end date as needed
dgs2_data = pdr.DataReader(dgs2_code,"fred", start=start_date, end=end_date)
dgs10_data = pdr.DataReader(dgs10_code,"fred", start=start_date, end=end_date)

# Join dataframes on date (assuming daily data)
joined_data = pd.DataFrame.join(dgs2_data, dgs10_data)
joined_data.dropna(inplace=True)  # Handle missing values

# Calculate spread (DGS10 - DGS2)
spread = joined_data[dgs10_code] - joined_data[dgs2_code]

# Find minimum spread after year 2000
min_spread_after_2000 = round(spread[spread.index.year > 2000].min(), 1)

# Print the minimum spread
print(f"Minimum value of (DGS10-DGS2) after 2000-01-01: {min_spread_after_2000}")



Minimum value of (DGS10-DGS2) after 2000-01-01: -1.1


**Inverted Yield Curve Discussion**

An inverted yield curve occurs when short-term interest rates (like DGS2) are higher than long-term rates (like DGS10). This is typically seen as a signal of a potential economic slowdown or recession. The rationale is that investors might demand a higher premium for holding long-term debt during uncertain economic times.

**Additional Considerations:**

-  **Market-Specific Analysis:**  The interpretation of the yield curve can vary depending on the specific market and economic conditions. Analyzing the historical behavior of the yield curve in your country/market of interest would be necessary to draw meaningful conclusions.
-  **Predictive Feature:** The spread (DGS10 - DGS2) might be a potential feature for economic forecasting models. However, its effectiveness would depend on the model design, historical data, and other economic indicators considered.



### Question 3. [Index] Which Index is better recently?

Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)


In [58]:
class StockIndexAnalyzer:

    @staticmethod
    def download_data(ticker_symbol, fields="Adj Close", start_date=None, end_date=None):
        """Downloads stock price data for a given symbol and timeframe.

        Args:
            ticker_symbol (str): The ticker symbol of the stock index.
            fields (str, optional): The data fields to download (default: "Adj Close").
            start_date (str, optional): The starting date for data download (default: None, downloads entire history).
            end_date (str, optional): The ending date for data download (default: None, downloads until today).

        Returns:
            pandas.DataFrame: The downloaded stock price data.
        """

        data = yf.download(ticker_symbol, start=start_date, end=end_date)[fields]
        return data

    @staticmethod
    def calc_growth_rates(data, time_delta=pd.Timedelta(days=0)):
        """Calculates growth rates for a given time delta.

        Args:
            data (pandas.DataFrame): The stock price data.
            time_delta (pandas.Timedelta): The time delta for growth rate calculation.

        Returns:
            pandas.Series: A Series containing growth rates for each date in the data.
        """
        # Ensure time_delta is positive (growth rate in the past)
        if time_delta.days < 0:
            time_delta *= -1
            
        # Calculate growth rates using shifted closing price
        shifted_data = data.shift(time_delta.days)

        growth_rates = ((data.iloc[-1] / shifted_data.iloc[-1]) - 1) * 100
        return growth_rates # Remove rows with NaN due to shifting

    @staticmethod
    def max_growth(growth_rate1, growth_rate2):
        """Compares two growth rates and returns the maximum one.

        Args:
            growth_rate1 (float): The first growth rate.
            growth_rate2 (float): The second growth rate.

        Returns:
            float: The maximum growth rate.
        """

        return round(max(growth_rate1, growth_rate2))

    @staticmethod
    def estimate_cagr(data):
        """Estimates the Compound Annual Growth Rate (CAGR) for a given time delta.

        Args:
            data (pandas.DataFrame): The stock price data.
            time_delta (pandas.Timedelta): The time delta for CAGR calculation.

        Returns:
            float: The estimated CAGR as a percentage.
        """

        # Assuming constant growth for simplicity (more accurate methods exist)
        # latest_date = data.index[-1]  # Get the latest date from the index
        n_periods = len(data) / 365  # Approximate number of periods (assuming daily data)        
        cagr_est = ((data.iloc[-1] / data.iloc[0])**(1/n_periods) - 1) * 100
        return cagr_est 
     


In [59]:
# Define start and end dates for 5Y growth calculation
start_date = "2019-04-09"
end_date = "2024-04-18"  # Adjust end date if needed

sp500_data = StockIndexAnalyzer.download_data(ticker_symbol="^GSPC", start_date=start_date, end_date=end_date)
ipc_data = StockIndexAnalyzer.download_data(ticker_symbol="^MXX", start_date=start_date, end_date=end_date)

# the data is already filter, use timedelta = 0
sp500_growth = StockIndexAnalyzer.calc_growth_rates(sp500_data)
ipc_growth = StockIndexAnalyzer.calc_growth_rates(ipc_data)

# Find the index with the highest growth (rounded to nearest integer %)
highest_growth = StockIndexAnalyzer.max_growth(sp500_growth, ipc_growth)

# Print the index with the highest 5-year growth
if sp500_growth == highest_growth:
    print(f"S&P 500 has the highest 5-year growth: {highest_growth}%")
else:
    print(f"IPC Mexico has the highest 5-year growth: {highest_growth}%")



[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed

S&P 500 has the highest 5-year growth: 0%


In [42]:
closing = yf.download("AAPL")["Close"]
today = closing.index[-1].date()  # Get the most recent closing date

[*********************100%%**********************]  1 of 1 completed


In [45]:
# Define start and end dates for different timeframes
ten_years_ago = today - pd.Timedelta(days=365 * 10)
twenty_years_ago = today - pd.Timedelta(days=365 * 20)

# Download data for each index
sp500 = StockIndexAnalyzer.download_data(ticker_symbol="^GSPC", start_date=start_date, end_date=end_date)
nasdaq = StockIndexAnalyzer.download_data(ticker_symbol="^IXIC", start_date=start_date, end_date=end_date)
dow_jones = StockIndexAnalyzer.download_data(ticker_symbol="^DJI", start_date=start_date, end_date=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [51]:
print(ten_years_ago)

2014-04-21


In [54]:

# Calculate 10-year and 20-year growth rates 
sp500_growth_10y =  StockIndexAnalyzer.calc_growth_rates(sp500, ten_years_ago) 
sp500_growth_20y = StockIndexAnalyzer.calc_growth_rates(sp500, twenty_years_ago) 

nasdaq_growth_10y = StockIndexAnalyzer.calc_growth_rates(nasdaq, ten_years_ago) 
nasdaq_growth_20y = StockIndexAnalyzer.calc_growth_rates(nasdaq, twenty_years_ago) 

dow_jones_growth_10y = StockIndexAnalyzer.calc_growth_rates(dow_jones, ten_years_ago) 
dow_jones_growth_20y = StockIndexAnalyzer.calc_growth_rates(dow_jones, twenty_years_ago) 


# Estimate CAGR (assuming constant growth for simplicity)
sp500_cagr_est = StockIndexAnalyzer.estimate_cagr(sp500)
nasdaq_cagr_est = StockIndexAnalyzer.estimate_cagr(nasdaq)
dow_jones_cagr_est = StockIndexAnalyzer.estimate_cagr(dow_jones)

# Print results
print("Index Name\t10-Year Growth (%)\t20-Year Growth (%)\tEstimated CAGR (%)")
print(f"S&P 500\t\t{sp500_growth_10y:.2f}\t\t\t{sp500_growth_20y:.2f}\t\t\t{sp500_cagr_est:.2f}")
print(f"Nasdaq\t\t{nasdaq_growth_10y:.2f}\t\t\t{nasdaq_growth_20y:.2f}\t\t\t{nasdaq_cagr_est:.2f}")
print(f"Dow Jones\t{dow_jones_growth_10y:.2f}\t\t\t{dow_jones_growth_20y:.2f}\t\t\t{dow_jones_cagr_est:.2f}")


Index Name	10-Year Growth (%)	20-Year Growth (%)	Estimated CAGR (%)
S&P 500		74.49			74.49			17.43
Nasdaq		98.29			98.29			21.84
Dow Jones	44.37			44.37			11.18
